In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
!gdown 'https://drive.google.com/uc?id=1BnKSXnaX8-sGzK1GNlAXve2pQ0Ql6CUz' -O '1.csv' #Thursday Working Hours Morning WebAttacks
!gdown 'https://drive.google.com/uc?id=1EdGVo3wQh52s7RTmSQ6uPGkknmjHcnL-' -O '2.csv' #Friday Working Hours - Afternoon -DDos
!gdown 'https://drive.google.com/uc?id=1vjb8V1vE1ZjKgJjuQ7jRa17yo00DMCGV' -O '3.csv' #Friday Working Hours - PortScan
!gdown 'https://drive.google.com/uc?id=13-EDXZDdxpogyVt4_UcSYJ-RN74kedCm' -O '4.csv' #Friday Working Hours - Morning
!gdown 'https://drive.google.com/uc?id=1yjc47co5wXZOsSC39Nb8KYcsWsphyyxh' -O '5.csv' #Monday Working Hours
!gdown 'https://drive.google.com/uc?id=1xJOrPX5QV3Ys3ZwtZR-wsYmQi3mVWvqO' -O '6.csv' #Thurday Working Hours Afternoon - Infilteration
!gdown 'https://drive.google.com/uc?id=143kbgfsr7TiRQ58rv6HfGca0nnf98T9t' -O '7.csv' #Tuesday Working Hours
!gdown 'https://drive.google.com/uc?id=1YxRK5X0paU3cXtxFpI1fzqf3F0Pph36t' -O '8.csv' #Wednesday Working Hours

Downloading...
From: https://drive.google.com/uc?id=1BnKSXnaX8-sGzK1GNlAXve2pQ0Ql6CUz
To: /content/1.csv
100% 52.0M/52.0M [00:00<00:00, 61.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1EdGVo3wQh52s7RTmSQ6uPGkknmjHcnL-
To: /content/2.csv
100% 77.1M/77.1M [00:00<00:00, 86.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1vjb8V1vE1ZjKgJjuQ7jRa17yo00DMCGV
To: /content/3.csv
100% 76.9M/76.9M [00:01<00:00, 42.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=13-EDXZDdxpogyVt4_UcSYJ-RN74kedCm
To: /content/4.csv
100% 58.3M/58.3M [00:00<00:00, 70.3MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1yjc47co5wXZOsSC39Nb8KYcsWsphyyxh
From (redirected): https://drive.google.com/uc?id=1yjc47co5wXZOsSC39Nb8KYcsWsphyyxh&confirm=t&uuid=3e571acf-0e78-47ed-9686-24dae7d80d0f
To: /content/5.csv
100% 177M/177M [00:02<00:00, 68.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1xJOrPX5QV3Ys3ZwtZR-wsYmQi3mVWvqO
To: /content/6.csv
100% 83.1M/83.1M [00

In [7]:
df1 = pd.read_csv('1.csv')
df2 = pd.read_csv('2.csv')

In [15]:
df = pd.read_csv('1.csv')
df.columns = [column.lstrip() for column in df.columns]
for i in range(2, 9):
  name = str(i) + '.csv'
  df1 = pd.read_csv(name)
  df1.columns =  [column.lstrip() for column in df1.columns]
  df = pd.concat([df, df1], ignore_index=True)

In [58]:
df.iloc[:, -1].value_counts()

,count
Label,
BENIGN,2273097
DoS,252661
PortScan,158930
DDoS,128027
Brute Force,13835
Web Attack,2180
Other,2013


In [27]:
#Shows the columns with null values
df[df.columns[df.isnull().sum().apply(lambda x: x > 0)]]

,Flow Bytes/s
0,1.740123e+02
1,2.122254e+02
2,0.000000e+00
3,1.087088e+02
4,1.576208e+06
...,...
2830738,8.194940e+03
2830739,1.376543e+06
2830740,4.512195e+05
2830741,4.272221e+02


In [29]:
flow_val = df['Flow Bytes/s'][df['Flow Bytes/s'] != np.inf].max()
flow_val

2071000000.0

In [31]:
#Replacing infinity with the next maximum value
df.loc[(df['Flow Bytes/s'] == np.inf), 'Flow Bytes/s'] = flow_val

In [32]:
#Replacing null values in Flow Bytes/s with median value
median_flow = df['Flow Bytes/s'].median()

df['Flow Bytes/s'] = df['Flow Bytes/s'].fillna(median_flow)

In [33]:
#Replacing infinity with null values
df.replace([np.inf, -np.inf], np.nan, inplace=True)

max_packets = df['Flow Packets/s'].max()
print(max_packets)
df['Flow Packets/s'] = df['Flow Packets/s'].fillna(max_packets)

4000000.0


Some columns like 'Bwd PSH Flags', 'Bwd URG Flags',' Fwd Avg Bytes/Bulk','Fwd Avg Packets/Bulk','Fwd Avg Bulk Rate','Bwd Avg Bytes/Bulk','Bwd Avg Packets/Bulk','Bwd Avg Bulk Rate' were found to contain only zeros so the columns can be dropped as they do not provide any new information.

In [34]:
df.drop(columns = ['Bwd PSH Flags', 'Bwd URG Flags','Fwd Avg Bytes/Bulk','Fwd Avg Packets/Bulk','Fwd Avg Bulk Rate','Bwd Avg Bytes/Bulk','Bwd Avg Packets/Bulk','Bwd Avg Bulk Rate'], inplace=True)

Several labels needs to be combined to create a bigger representation, so that the class imbalance becomes less prominent.

In [52]:
df['Label'] = df['Label'].replace(['DoS Hulk', 'DoS GoldenEye', 'DoS slowloris', 'DoS Slowhttptest'], 'DoS')

In [53]:
df['Label'] = df['Label'].replace(['Web Attack � Brute Force', 'Web Attack � XSS', 'Web Attack � Sql Injection'], 'Web Attack')

In [54]:
df['Label'] = df['Label'].replace(['FTP-Patator', 'SSH-Patator'], 'Brute Force')

In [57]:
df['Label'] = df['Label'].replace(['Bot', 'Infiltration', 'Heartbleed'], 'Other')

In [59]:
sampled_df = df[df['Label'].isin(['Other', 'Web Attack', 'Brute Force'])]

In [60]:
sampled_df['Label'].value_counts()

,count
Label,
Brute Force,13835
Web Attack,2180
Other,2013


In [61]:
benign_df = df[df['Label'] == 'BENIGN'].sample(100000)
dos_df = df[df['Label'] == 'DoS'].sample(80000)
port_df = df[df['Label'] == 'PortScan'].sample(75000)
ddos_df = df[df['Label'] == 'DDoS'].sample(65000)

In [62]:
sampled_df = pd.concat([sampled_df, benign_df, dos_df, port_df, ddos_df], ignore_index=True)

In [ ]:
correlation = sampled_df.select_dtypes(include='float64').corr().abs()

In [67]:
sampled_df.to_csv('out.csv', encoding='utf-8', index=False, header=True)